# W261 Final Project ETL for Development Sample

### Notebook Set-Up

In [1]:
# imports
import time
import numpy as np
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.feature import CountVectorizer
from pyspark.sql import DataFrame

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]
PWD

'/media/notebooks'

In [4]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "w261FinalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

__`REMINDER:`__ If you are running this notebook on the course docker container, you can monitor the progress of your jobs using the Spark UI at: http://localhost:4040/jobs/

## Load Data

In [5]:
original_trainRDD = sc.textFile('data/train.txt')
original_testRDD = sc.textFile('data/test.txt')

## Transformation

### Sample

In [6]:
#change the seed for a different sample
sampleRDD1, sampleRDD2 = original_trainRDD.randomSplit([0.9999,0.0001], seed = 1)
sampleRDD2.cache()

PythonRDD[4] at RDD at PythonRDD.scala:49

In [7]:
ncol = len(sampleRDD2.take(1)[0].split('\t'))
nrow = sampleRDD2.count()
print("This sample contains", str(nrow), "rows.")

This sample contains 4478 rows.


In [8]:
print("This sample contains", str(ncol), "columns.")

This sample contains 40 columns.


In [9]:
def avgFeatures(line):
    
    count = 0
    feats = line.split('\t')[1:]
    
    for feat in feats:
        if feat != '':
            count += 1

    return count

print("There is an average of", str(round(sampleRDD2.map(avgFeatures).mean(),2)), "populated features per observation.")
sampleRDD2.map(avgFeatures).mean()

There is an average of 33.53 populated features per observation.


33.5288075033497

# Put in wide, sparse feature format

In [95]:
def parseCV(line):
    """
    Map record_csv_string --> (features, label)
    """

    # start of categorical features
    col_start = 14
    
    raw_values = line.split('\t')
    label = int(raw_values[0])
    
    # ignore numerics to start
    #numerical_values = list(pd.Series(raw_values[1:14]).apply(pd.to_numeric))
    numericals = []
    for idx, value in enumerate(raw_values[1:col_start]):
        if value != '':
            numericals.append('n' + str(idx) + '_' + str(value))
            
    
    categories = []
    for idx, value in enumerate(raw_values[col_start:]):
        if value != '':
            categories.append('c'+ str(idx) + '_' + str(value))

    return Row(label=label, raw=numericals + categories)


def vectorizeCV(DF):
    
    vectorizer = CountVectorizer()
    cv = CountVectorizer(inputCol="raw", outputCol="features", binary=True)
    
    model = cv.fit(DF)
    result = model.transform(DF)
    num_feats = cv.getVocabSize()
    
    return result


parsedDF = sampleRDD2.map(parseCV).toDF().cache()
vectorizedDF = vectorizeCV(parsedDF)

In [96]:
vectorizedDF.show(truncate=True)

+-----+--------------------+--------------------+
|label|                 raw|            features|
+-----+--------------------+--------------------+
|    0|[n1_4, n2_50, n3_...|(30946,[0,1,2,4,5...|
|    0|[n1_12, n2_20, n3...|(30946,[0,1,2,5,1...|
|    1|[n1_1, n2_1, n4_9...|(30946,[0,1,6,7,1...|
|    0|[n0_8, n1_17, n3_...|(30946,[0,1,4,12,...|
|    1|[n0_6, n1_1, n2_7...|(30946,[0,1,2,4,1...|
|    1|[n1_99, n2_1, n3_...|(30946,[1,2,4,10,...|
|    0|[n0_3, n1_21, n2_...|(30946,[0,1,4,8,1...|
|    0|[n1_2, n2_20, n3_...|(30946,[0,1,3,5,8...|
|    0|[n0_0, n1_144, n4...|(30946,[0,2,3,4,5...|
|    0|[n1_0, n2_5, n4_3...|(30946,[0,2,3,6,1...|
|    0|[n0_0, n1_1, n2_4...|(30946,[0,1,2,3,5...|
|    0|[n0_9, n1_5, n2_1...|(30946,[0,2,3,6,9...|
|    0|[n1_323, n2_2, n3...|(30946,[1,2,14,16...|
|    0|[n0_0, n1_424, n3...|(30946,[0,1,2,4,6...|
|    0|[n0_0, n1_13, n2_...|(30946,[0,1,2,5,6...|
|    0|[n1_180, n2_6, n3...|(30946,[1,2,8,14,...|
|    0|[n1_126, n2_2, n3...|(30946,[0,2,4,6,8...|


In [97]:
vectorizedRDD = vectorizedDF.select(['label', 'features']).rdd.cache()
vectorizedRDD.take(1)

[Row(label=0, features=SparseVector(30946, {0: 1.0, 1: 1.0, 2: 1.0, 4: 1.0, 5: 1.0, 7: 1.0, 10: 1.0, 20: 1.0, 32: 1.0, 122: 1.0, 155: 1.0, 174: 1.0, 214: 1.0, 364: 1.0, 369: 1.0, 495: 1.0, 504: 1.0, 632: 1.0, 635: 1.0, 869: 1.0, 1895: 1.0, 2117: 1.0, 2169: 1.0, 2254: 1.0, 2774: 1.0, 6282: 1.0, 11894: 1.0, 13231: 1.0, 14089: 1.0, 18906: 1.0, 20114: 1.0, 24029: 1.0, 25433: 1.0, 25595: 1.0}))]

In [100]:
# feature data struct tester
num_feats = vectorizedRDD.take(1)[0][1].size
num_feats

30946

In [13]:
vectorizedRDD.map(lambda x: x[0]).mean()

0.25949084412684253

## test prediction making

In [18]:
#sc.addPyFile("./fm_function_tester.py")
#import fm_function_tester as fm
#from fm_function_tester import *

(4478, 2)

In [101]:
# initialize weights
b_br = sc.broadcast(0.0)
w_br = sc.broadcast(np.zeros(num_feats, dtype=np.float16))
k_br = sc.broadcast(2)
V_br = sc.broadcast(np.random.normal(0.0, 0.5, (num_feats, k_br.value)))

In [102]:
w_br.value.shape

(30946,)

In [106]:
import numpy as np
from pyspark.ml.feature import CountVectorizer

def predictCTR(pair):
    """
        Compute the predicted probability AND return the gradient (?)
        Args:
            pair - records are in (label, sparse feature set) format
        Broadcast:
            b - bias term (scalar)
            w - linear weight vector (array)
            k - number of factors (def=2)
            V - factor matrix of size (d dimensions, k=2 factors)
        Returns:
            predRDD - pair of ([label, predicted probability], feature set)
    """
    
    feats = pair[1]
    
    # start with linear weight dot product
    linear_sum = 0.0
    for i in feats.indices:
        i = int(i)
        linear_sum += w_br.value[i]*feats[i]

    # factor matrix interaction sum
    factor_sum = 0.0
    lh_factor = 0.0
    rh_factor = 0.0
    
    for f in range(0, k_br.value):
        
        for i in feats.indices:
            i = int(i)
            lh_factor += V_br.value[i][f]*feats[i]
            rh_factor += (V_br.value[i][f]**2) * (feats[i]**2)
        
        factor_sum += (lh_factor**2 - rh_factor)
    factor_sum = 0.5 * factor_sum
    
    preProb = b_br.value + linear_sum + factor_sum
    
    prob = 1.0 / (1 + np.exp(-preProb))
    
    return (pair[0], prob)

In [107]:
predRDD = vectorizedRDD.map(predictCTR).cache()
predRDD.take(2)

[(0, 0.9999351260667665), (0, 0.0001032597463594804)]